In [2]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import pandas as pd
import h3
from modules.config import PATH_SCENARIOS_REDUCED, H3_RESOLUTION, PERIOD_DURATION, PATH_DISTANCES
from modules.distances import calc_distance_haversine

## Distances

In [4]:
avg_distance_l1 = 0.8259763178117513
avg_distance = avg_distance_l1 * h3.edge_length(H3_RESOLUTION, unit='km')

In [5]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [6]:
def calc_distance(hexes):
    if hexes[0] == hexes[1]:
        return avg_distance
    return calc_distance_haversine(
        h3.h3_to_geo(hexes[0]),
        h3.h3_to_geo(hexes[1])
        )

In [7]:
distances = pd.DataFrame(index=pd.MultiIndex.from_product([hex_ids, hex_ids]))
distances.index = distances.index.rename(['start_hex_id', 'end_hex_id'])
distances['distance'] = distances.index.map(calc_distance)

### Profit
Maybe move some to config?

In [8]:
vehicle_speed = { # in km/min
    'kick_scooter':  (0.1117 + 0.0497)/2, 
    'bicycle': 0.0497,
    'car': 0.1117,
}

vehicle_profit_min = { # euro / min
    'kick_scooter': 0.19, # wrong data
    'bicycle': 0.1 / 3,
    'car': 0.29,
}

vehicle_profit_km = {
        key: p_m/vehicle_speed[key] 
        for key,p_m in vehicle_profit_min.items() 
}

In [9]:
distances['profit_kick_scooter'] = distances['distance'].map(
    lambda distance: (distance * vehicle_profit_km['kick_scooter'])
)
distances['profit_bicycle'] = distances['distance'].map(
    lambda distance: (distance * vehicle_profit_km['bicycle'])
)
distances['profit_car'] = distances['distance'].map(
    lambda distance: (distance * vehicle_profit_km['car'])
)

### Costs

In [10]:

relocation_driver_salary = 14 # euro/hour
vehicle_rel_at_once = { # relocations at once, e.g. in a truck for moped/bicles
    'kick_scooter': 40,
    'bicycle': 20,
    'car': 1,
}

vehicle_cost_min = {
    key: (relocation_driver_salary / 60 / rel_at_once) 
    for key, rel_at_once in vehicle_rel_at_once.items()
}

vehicle_cost_km = {
    key: c_m/vehicle_speed[key] 
    for key,c_m in vehicle_cost_min.items() 
}
vehicle_cost_km

{'kick_scooter': 0.07228418009087155,
 'bicycle': 0.2347417840375587,
 'car': 2.0889286780065652}

In [11]:
distances['cost_kick_scooter'] = distances['distance'].map(
    lambda distance: (distance * vehicle_cost_km['kick_scooter'])
)
distances['cost_bicycle'] = distances['distance'].map(
    lambda distance: (distance * vehicle_cost_km['bicycle'])
)
distances['cost_car'] = distances['distance'].map(
    lambda distance: (distance * vehicle_cost_km['car'])
)

### Parking costs

In [12]:
vehicle_parking_cost = { # eur/2h
    'kick_scooter': 0.4,
    'car': 2,
    'bicycle': 0.1,
}

vehicle_maximum_distance_cap = {
    'kick_scooter': 4,
    'bicycle': 5,
    'car': 100,
} # km


In [13]:
distances = distances.reset_index()

same_hexagon = distances['start_hex_id'] == distances['end_hex_id']
distances.loc[same_hexagon, 'cost_kick_scooter'] = vehicle_parking_cost['kick_scooter'] * PERIOD_DURATION
distances.loc[same_hexagon, 'cost_car'] = vehicle_parking_cost['car'] * PERIOD_DURATION
distances.loc[same_hexagon, 'cost_bicycle'] = vehicle_parking_cost['bicycle'] * PERIOD_DURATION

distances = distances.set_index(['start_hex_id', 'end_hex_id'])

In [14]:
os.makedirs(os.path.dirname(PATH_DISTANCES), exist_ok=True)
distances.to_pickle(PATH_DISTANCES)